# Livestock data generation

In [3]:
SCENARIO_DURATIONS = [91, 182, 365] # days

# residue consumption by cattle in units of kg per cow (total across scenario considered)
DM_CONSUMPTIONS_CATTLE = [11 * s for s in SCENARIO_DURATIONS] 

#TODO goats
#TODO sheep

MIN_HERD_SIZE = 50 # min number of cattle in a 'pixel' which we'll consider serving
GRAZE_EFF = 0.5 # percent of dry matter consumable by grazing

In [4]:
# Allow access to Python imports from higher level folder
import sys
sys.path.append('..')

# Define directory locations
TEST_DIR = '../../../data/livestock/CGIAR/CATTLE/'
# IN_SHAPE_DIR = '../test_data/usa/in_shape/'

# ALLFED libs
from raster.conversions import raster_to_features
from geometry.project import project_features_to_utm, project_features_from_utm
from features.io import write_features

In [27]:
# Load livestock and convert to features
COW_PATH = TEST_DIR + 'Glb_Cattle_CC2006_AD_resampled_01.tif'
cow_features = raster_to_features(COW_PATH)

In [34]:
# Load livestock and convert to features
HD_COW_PATH = TEST_DIR + 'Glb_Cattle_CC2006_AD_resampled_02.tif'
hd_cow_features = raster_to_features(HD_COW_PATH)

In [29]:
sum([c.data['value'] for c in cow_features]) * 92

1239682037.581188

In [35]:
sum([c.data['value'] for c in hd_cow_features]) * 490

1629858347.653659

In [24]:
53759475 * 23

1236467925

In [25]:
13474804 * 92

1239681968

In [36]:
3326241 * 490

1629858090

In [26]:
1937289062 * 0.633

1226303976.246

In [32]:
1936835691 * 0.836

1619194637.6759999

In [37]:
2.479240569868125 * 22639088

56127745.43441463

In [42]:
1656164153.793699 * 0.855

1416020351.4936125

In [21]:
len(cow_features)

1106291

In [5]:
# Add consumption data to livestock
# Pixel values are head per sq. km.
for lf in cow_features:
    lf.update_data('type', 'cattle')
    lf.update_data('head', lf.data['value'] * lf.data['pixel_size'])
    lf.update_data('demand_kg', lf.data['head'] * DM_CONSUMPTION_CATTLE) # total demand in kg
    lf.update_data('remaining', lf.data['demand_kg'] )
    lf.update_data('node_type', 'demand')
    lf.update_data('grazed_{}'.format(int(GRAZE_EFF * 100)), 0)

In [6]:
# Filter out cattle pixels which don't meet herd size requirement
cow_features = [lf for lf in filter(lambda l: l.data['head'] >= MIN_HERD_SIZE, cow_features)]

In [7]:
# Write to file
write_features(cow_features, IN_SHAPE_DIR + 'livestock_points.shp')